# Assignment 6: Part B, Pytorch Lightning

In [2]:
!pip install pytorch_lightning --quiet

     |████████████████████████████████| 849kB 9.2MB/s 
     |████████████████████████████████| 276kB 27.4MB/s 
     |████████████████████████████████| 112kB 32.4MB/s 
     |████████████████████████████████| 829kB 27.6MB/s 
     |████████████████████████████████| 184kB 51.2MB/s 
     |████████████████████████████████| 1.3MB 55.0MB/s 
     |████████████████████████████████| 296kB 56.5MB/s 
     |████████████████████████████████| 143kB 56.0MB/s 


In [3]:
import os
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader,random_split
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
import numpy as np

In [4]:
print(torch.__version__)

1.8.1+cu101


In [5]:
#defining the model
class smallAndSmartModel(pl.LightningModule):

    def __init__(self):
        #this is the init function where we will defnine the architecture
        super(smallAndSmartModel, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1,28,kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2))
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(28,10,kernel_size=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2))
        self.dropout1=torch.nn.Dropout(0.25)
        self.fc1=torch.nn.Linear(250,18)
        self.dropout2=torch.nn.Dropout(0.08)
        self.fc2=torch.nn.Linear(18,10)
        #we are also defing some variable for counting purposes
        self.valTotal=0
        self.valCorrect=0
        self.trainTotal=0
        self.trainCorrect=0


    def prepare_data(self):
        
        # This contains the manupulation on data that needs to be done only once such as downloading it

        #download the MNIST dataset
        MNIST(os.getcwd(), train=True, download =True)
        MNIST(os.getcwd(), train=False, download =True)
        
        # See here I have set download to false as it is already downloaded
        mnist_train=MNIST(os.getcwd(), train=True, download =False,transform=transforms.ToTensor())

        #dividing into validation and training set
        self.train_set, self.val_set= random_split(mnist_train,[55000,5000])
    
    def train_dataloader(self):
        # REQUIRED
        # This is an essential function. Needs to be included in the code
               
        return DataLoader(self.train_set,batch_size=128,num_workers=32)
        
    def val_dataloader(self):
        # OPTIONAL
        #loading validation dataset
        return DataLoader(self.val_set, batch_size=128,num_workers=32)

    def test_dataloader(self):
        # OPTIONAL
        # loading test dataset
        return DataLoader(MNIST(os.getcwd(), train=False, download=False, transform=transforms.ToTensor()), batch_size=128,num_workers=32)

    def forward(self,x):
        #evaluating the batch data as it moves forward in the netowrk
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.dropout1(x)
        x=torch.relu(self.fc1(x.view(x.size(0), -1)))
        x=F.leaky_relu(self.dropout2(x))

        return F.softmax(self.fc2(x))

    def training_step(self,batch,batch_idx):
        #REQUIRED

        #extracting input and output from the batch
        x,labels=batch

        #doing a forward pass
        pred=self.forward(x)

        self.trainCorrect+=pred.argmax(dim=1).eq(labels).sum().item()
        self.trainTotal+=len(labels)
        #calculating the loss
        train_loss = F.cross_entropy(pred, labels)

        #logs
        logs={"train_loss": train_loss}

        output={
            #REQUIRED: It ie required for us to return "loss"
            "loss": train_loss,
            #optional for logging purposes
            "log": logs
        }

        return output

    def on_epoch_start(self):
        # OPTIONAL
        # runs on the begining of an epoch
        self.valTotal=0
        self.valCorrect=0
        self.testTotal=0
        self.testCorrect=0
        self.trainTotal=0
        self.trainCorrect=0

    def validation_step(self, batch, batch_nb):
        # OPTIONAL
        # The code that runs as we forward pass a validation batch
        x, y = batch
        y_hat = self(x)
        self.valCorrect+=y_hat.argmax(dim=1).eq(y).sum().item()
        self.valTotal+=len(y)
        return {'val_loss': F.cross_entropy(y_hat, y)}

    def validation_epoch_end(self, outputs):
        # OPTIONAL
        # The code that runs as a validation epoch finished
        # Used for metric evaluation
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        print("Number of Correctly identified Validation Images {} from aset of {}. \nAccuracy= {} ".format(self.valCorrect,self.valTotal,self.valCorrect/self.valTotal))
        
        tensorboard_logs = {'val_loss': avg_loss,"Accuracy": self.valCorrect/self.valTotal}
        return {'val_loss': avg_loss, 'log': tensorboard_logs}

    def test_step(self, batch, batch_nb):
        # OPTIONAL
        # The code that runs as we forward pass a test batch

        x, y = batch
        y_hat = self(x)
        self.testCorrect+=y_hat.argmax(dim=1).eq(y).sum().item()
        self.testTotal+=len(y)
        return {'test_loss': F.cross_entropy(y_hat, y)}


    def training_epoch_end(self,outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        tensorboard_logs = {'loss': avg_loss,"Accuracy": self.trainCorrect/self.trainTotal}
        print("Number of Correctly identified Training Set Images {} from a set of {}. \nAccuracy= {} ".format(self.trainCorrect,self.trainTotal,self.trainCorrect/self.trainTotal))
        return {'loss': avg_loss, 'log': tensorboard_logs}

   
    def test_epoch_end(self, outputs):
        # OPTIONAL
        # The code that runs as a validation epoch finished
        # Used for metric evaluation
        print("Number of Correctly identified Testing Images {} from aset of {}. \nAccuracy= {} ".format(self.testCorrect,self.testTotal,self.testCorrect/self.testTotal))

        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        logs = {'test_loss': avg_loss}
        return {'test_loss': avg_loss, 'log': logs, 'progress_bar': logs}

    def configure_optimizers(self):
        # REQUIRED
        # Can return multiple optimizers and learning_rate schedulers
        return torch.optim.Adam(self.parameters(), lr=0.01)


In [6]:
# Using the Lightning trainer and specifing the requied parameters as arguments
myTrainer=pl.Trainer(gpus=1,checkpoint_callback=False)

myModel=smallAndSmartModel()
myTrainer.fit(myModel)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting /content/MNIST/raw/train-images-idx3-ubyte.gz to /content/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting /content/MNIST/raw/train-labels-idx1-ubyte.gz to /content/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting /content/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



...

Number of Correctly identified Validation Images 4196 from aset of 5000. 
Accuracy= 0.8392 
Number of Correctly identified Training Set Images 41508 from a set of 55000. 
Accuracy= 0.7546909090909091 


Number of Correctly identified Validation Images 4163 from aset of 5000. 
Accuracy= 0.8326 
Number of Correctly identified Training Set Images 43322 from a set of 55000. 
Accuracy= 0.7876727272727273 


Number of Correctly identified Validation Images 4350 from aset of 5000. 
Accuracy= 0.87 


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fc26e1f7c20>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1291, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


1

In [7]:
myTrainer=pl.Trainer(gpus=1, max_epochs=10)

model=smallAndSmartModel()
myTrainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type       | Params
----------------------------------------
0 | layer1   | Sequential | 728   
1 | layer2   | Sequential | 1.1 K 
2 | dropout1 | Dropout    | 0     
3 | fc1      | Linear     | 4.5 K 
4 | dropout2 | Dropout    | 0     
5 | fc2      | Linear     | 190   
----------------------------------------
6.6 K     Trainable params
0         Non-trainable params
6.6 K     Total params
0.026     Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Number of Correctly identified Validation Images 22 from aset of 256. 
Accuracy= 0.0859375 


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The validation_epoch_end should not return anything as of 9.1. To log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.
# log on a step or aggregate epoch metric to the logger and/or progress bar (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


...

Number of Correctly identified Validation Images 4761 from aset of 5000. 
Accuracy= 0.9522 
Number of Correctly identified Training Set Images 49359 from a set of 55000. 
Accuracy= 0.8974363636363636 


Number of Correctly identified Validation Images 4729 from aset of 5000. 
Accuracy= 0.9458 
Number of Correctly identified Training Set Images 49076 from a set of 55000. 
Accuracy= 0.8922909090909091 


Number of Correctly identified Validation Images 4731 from aset of 5000. 
Accuracy= 0.9462 
Number of Correctly identified Training Set Images 48705 from a set of 55000. 
Accuracy= 0.8855454545454545 


Number of Correctly identified Validation Images 4752 from aset of 5000. 
Accuracy= 0.9504 



1